In [13]:
import pandas as pd
import numpy as np

import archs4py as a4
import importlib

importlib.reload(a4)
a4.reload()

In [4]:
processed_gsm = a4.db.processed_gsms()
srr = a4.sra.get_gsms()

In [14]:
new_samples = a4.geo.scan_platforms(srr, processed_gsm)


  0%|          | 0/22 [00:00<?, ?it/s]22-Oct-2021 20:34:12 INFO GEOparse - Parsing downloads/soft/GPL24676.soft.gz: 


18287


  5%|▍         | 1/22 [01:19<27:47, 79.41s/it]22-Oct-2021 20:35:31 INFO GEOparse - Parsing downloads/soft/GPL24247.soft.gz: 


148933


  9%|▉         | 2/22 [05:18<57:45, 173.28s/it]22-Oct-2021 20:39:30 INFO GEOparse - Parsing downloads/soft/GPL21626.soft.gz: 


...

 27%|██▋       | 6/22 [06:55<11:34, 43.38s/it]22-Oct-2021 20:41:07 INFO GEOparse - Parsing downloads/soft/GPL21493.soft.gz: 


3004


 32%|███▏      | 7/22 [07:19<09:16, 37.13s/it]22-Oct-2021 20:41:31 INFO GEOparse - Parsing downloads/soft/GPL21103.soft.gz: 


18455


 36%|███▋      | 8/22 [08:55<13:04, 56.06s/it]22-Oct-2021 20:43:08 INFO GEOparse - Parsing downloads/soft/GPL19057.soft.gz: 


In [ ]:
maxlistid = a4.db.get_max_id()

gsm_meta = new_samples
gsm_meta["listid"] = list(range(maxlistid+1, maxlistid+gsm_meta.shape[0]+1))
sample_mapping = gsm_meta.loc[:, ["gse", "gsm", "sra", "listid"]]
sample_mapping = sample_mapping.sort_values("gse").reset_index(drop=True)

a4.db.add_mapping(sample_mapping)
a4.db.add_sequencing_jobs(new_samples)

,gse,gsm,sra,listid
0,GSE100064,GSM2670741,SRR5687234,1583729
1,GSE100064,GSM2670740,SRR5687233,1583728
2,GSE100391,GSM2680090,SRR5750314,1583730
3,GSE100391,GSM2680182,SRR5750406,1647514
4,GSE100391,GSM2680181,SRR5750405,1647513
...,...,...,...,...
290052,GSE98548,GSM2598508,SRR5508144,1556867
290053,GSE98548,GSM2598507,SRR5508143,1556866
290054,GSE98548,GSM2598506,SRR5508142,1556865
290055,GSE98548,GSM2598505,SRR5508141,1556864


In [9]:
from tqdm import tqdm

def test(i):
    print(i)

for i in tqdm(list(range(10))):
    test(i)


100%|██████████| 10/10 [00:00<00:00, 37888.93it/s]

0
1
2
3
4
5
6
7
8
9
